# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
#Importing libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#Loading data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql_table("Messages",engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X.to_csv("x.csv")

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    inputs:
    messages
       
    Returns:
    messages split into multiple words
    """
    #Converting everything into lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #Tokenizing the words
    token = word_tokenize(text)
    
    #Normalizing the word tokens and removing stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized_messages = [normlizer.stem(word) for word in token if word not in stop_words]
    #print(normlized_messages)
    return normlized_messages

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline_1 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 23)

np.random.seed(39)
pipeline_1.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def evaluation_metrics(ArrayActual, ArrayPredicted, col_names):
    """Evalute the metrics/KPIs of the Machine Learning(ML) pipeline model
    
    inputs:
    ArrayActual: array. Array containing the real labels.
    ArrayPredicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayPredicted fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayActual and ArrayPredicted labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayActual[:, i], ArrayPredicted[:, i])
        precision = precision_score(ArrayActual[:, i], ArrayPredicted[:, i], average='micro')
        recall = recall_score(ArrayActual[:, i], ArrayPredicted[:, i], average='micro')
        f1 = f1_score(ArrayActual[:, i], ArrayPredicted[:, i], average='micro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics   

In [9]:
# Calculate evaluation metrics for training set
Y_train_pred = pipeline_1.predict(X_train)
col_names = list(Y.columns.values)

print(evaluation_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.989981   0.989981  0.989981  0.989981
request                 0.986827   0.986827  0.986827  0.986827
offer                   0.998779   0.998779  0.998779  0.998779
aid_related             0.986064   0.986064  0.986064  0.986064
medical_help            0.987336   0.987336  0.987336  0.987336
medical_products        0.991150   0.991150  0.991150  0.991150
search_and_rescue       0.994507   0.994507  0.994507  0.994507
security                0.994812   0.994812  0.994812  0.994812
military                0.995575   0.995575  0.995575  0.995575
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.995016   0.995016  0.995016  0.995016
food                    0.995016   0.995016  0.995016  0.995016
shelter                 0.992218   0.992218  0.992218  0.992218
clothing                0.997559   0.997559  0.997559  0.997559
money                   0.995982   0.995

In [10]:
# Calculate evaluation metrics for test set
Y_test_pred = pipeline_1.predict(X_test)

evaluation_metrics_test = evaluation_metrics(np.array(Y_test), Y_test_pred, col_names)
print(evaluation_metrics_test)

                        Accuracy  Precision    Recall        F1
related                 0.806530   0.806530  0.806530  0.806530
request                 0.879768   0.879768  0.879768  0.879768
offer                   0.996186   0.996186  0.996186  0.996186
aid_related             0.754654   0.754654  0.754654  0.754654
medical_help            0.924016   0.924016  0.924016  0.924016
medical_products        0.951327   0.951327  0.951327  0.951327
search_and_rescue       0.971315   0.971315  0.971315  0.971315
security                0.981233   0.981233  0.981233  0.981233
military                0.969942   0.969942  0.969942  0.969942
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.955905   0.955905  0.955905  0.955905
food                    0.925694   0.925694  0.925694  0.925694
shelter                 0.926305   0.926305  0.926305  0.926305
clothing                0.986421   0.986421  0.986421  0.986421
money                   0.976350   0.976

In [11]:
# Calculation the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
#Defining performance metric for use in grid search scoring object
def performance_metric(Y_Actual, Y_Predicted):
    """Median F1 score for all classifiers
    
    inputs:
    Y_Actual: array. Array containing ArrayActual labels.
    Y_Predicted: array. Array containing ArrayPredicted labels.
        
    Routputs:
    Median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_Predicted)[1]):
        f1 = f1_score(np.array(Y_Actual)[:, i], Y_Predicted[:, i], average = 'micro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [13]:
##delete
scorer = make_scorer(performance_metric)

In [14]:
#Performing grid search method

parameters = {'vect__min_df': [10],
              'tfidf__use_idf':[True],
              'clf__estimator__n_estimators':[50], 
              'clf__estimator__min_samples_split':[10]}

scorer = make_scorer(performance_metric)
cv = GridSearchCV(pipeline_1, param_grid = parameters, scoring = scorer, verbose = 10)

#Finding best parameters
np.random.seed(74)
model_1 = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__min_df=10 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__min_df=10, score=0.9598718339945072, total= 1.9min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__min_df=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__min_df=10, score=0.9653646628013427, total= 1.9min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__min_df=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__min_df=10, score=0.9604821483063778, total= 1.9min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  6.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  6.9min finished


In [15]:
#Checking grid search results
model_1.cv_results_

{'mean_fit_time': array([ 101.43601855]),
 'std_fit_time': array([ 0.75391672]),
 'mean_score_time': array([ 12.41139007]),
 'std_score_time': array([ 0.14640807]),
 'param_clf__estimator__min_samples_split': masked_array(data = [10],
              mask = [False],
        fill_value = ?),
 'param_clf__estimator__n_estimators': masked_array(data = [50],
              mask = [False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True],
              mask = [False],
        fill_value = ?),
 'param_vect__min_df': masked_array(data = [10],
              mask = [False],
        fill_value = ?),
 'params': [{'clf__estimator__min_samples_split': 10,
   'clf__estimator__n_estimators': 50,
   'tfidf__use_idf': True,
   'vect__min_df': 10}],
 'split0_test_score': array([ 0.95987183]),
 'split1_test_score': array([ 0.96536466]),
 'split2_test_score': array([ 0.96048215]),
 'mean_test_score': array([ 0.96190622]),
 'std_test_score': array([ 0.00245815]),
 'rank_test_score'

In [16]:
#Evaluating top mean score
np.max(model_1.cv_results_['mean_test_score'])

0.96190621503407592

In [17]:
#Displaying parameters for top mean score
model_1.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': True,
 'vect__min_df': 10}

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
#Evaluating metrics for the test dataset
tuned_pred_test = model_1.predict(X_test)

evaluation_metrics_test_tuned = evaluation_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(evaluation_metrics_test_tuned)

                        Accuracy  Precision    Recall        F1
related                 0.819805   0.819805  0.819805  0.819805
request                 0.891669   0.891669  0.891669  0.891669
offer                   0.996186   0.996186  0.996186  0.996186
aid_related             0.782270   0.782270  0.782270  0.782270
medical_help            0.927983   0.927983  0.927983  0.927983
medical_products        0.953158   0.953158  0.953158  0.953158
search_and_rescue       0.972536   0.972536  0.972536  0.972536
security                0.981385   0.981385  0.981385  0.981385
military                0.971315   0.971315  0.971315  0.971315
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.964907   0.964907  0.964907  0.964907
food                    0.953464   0.953464  0.953464  0.953464
shelter                 0.944614   0.944614  0.944614  0.944614
clothing                0.987031   0.987031  0.987031  0.987031
money                   0.975893   0.975

In [19]:
#Chceking summary of the 1st model
evaluation_metrics_test.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.944461,0.944461,0.944461,0.944461
std,0.056373,0.056373,0.056373,0.056373
min,0.754654,0.754654,0.754654,0.754654
25%,0.931111,0.931111,0.931111,0.931111
50%,0.957431,0.957431,0.957431,0.957431
75%,0.982797,0.982797,0.982797,0.982797
max,1.000000,1.000000,1.000000,1.000000


In [20]:
#Chceking summary of the tuned model
evaluation_metrics_test_tuned.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.950069,0.950069,0.950069,0.950069
std,0.050733,0.050733,0.050733,0.050733
min,0.782270,0.782270,0.782270,0.782270
25%,0.946674,0.946674,0.946674,0.946674
50%,0.964525,0.964525,0.964525,0.964525
75%,0.982911,0.982911,0.982911,0.982911
max,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report

pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

np.random.seed(23)
model_2 = pipeline2.fit(X_train, Y_train)

In [22]:
#Evaluating metrics for the test dataset
tuned_pred_test_2 = model_2.predict(X_test)

evaluation_metrics_test_tuned_2 = evaluation_metrics(np.array(Y_test), tuned_pred_test_2, col_names)

print(evaluation_metrics_test_tuned_2)

                        Accuracy  Precision    Recall        F1
related                 0.768691   0.768691  0.768691  0.768691
request                 0.882057   0.882057  0.882057  0.882057
offer                   0.994965   0.994965  0.994965  0.994965
aid_related             0.760146   0.760146  0.760146  0.760146
medical_help            0.929966   0.929966  0.929966  0.929966
medical_products        0.957431   0.957431  0.957431  0.957431
search_and_rescue       0.973756   0.973756  0.973756  0.973756
security                0.980012   0.980012  0.980012  0.980012
military                0.969942   0.969942  0.969942  0.969942
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.962771   0.962771  0.962771  0.962771
food                    0.947360   0.947360  0.947360  0.947360
shelter                 0.945224   0.945224  0.945224  0.945224
clothing                0.987489   0.987489  0.987489  0.987489
money                   0.978181   0.978

In [23]:
#Chceking summary of the 3rd model
evaluation_metrics_test_tuned_2.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.946822,0.946822,0.946822,0.946822
std,0.058451,0.058451,0.058451,0.058451
min,0.760146,0.760146,0.760146,0.760146
25%,0.944461,0.944461,0.944461,0.944461
50%,0.964297,0.964297,0.964297,0.964297
75%,0.983102,0.983102,0.983102,0.983102
max,1.000000,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [24]:
#Pickle best model
pickle.dump(model_1, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.